Data Wrangling

In [ ]:
# Imports
import glob
import shutil
from pathlib import Path

import matplotlib.pyplot as plt
import random

import kagglehub

import tensorflow as tf
from tensorflow.keras import layers, Sequential

import shutil

import pandas as pd
from collections import Counter

Data Download

In [ ]:
# Download latest version
path = kagglehub.dataset_download("paultimothymooney/breast-histopathology-images")

print("Path to dataset files:", path)

2. Data re-structuring

Right now, the data exists in a set of numbered directories, one for each patient. In each patient directory there are two sub-directories 0 and 1 containing IDC negative and IDC positive images. We need to collect all of the IDC negative images and IDC positive images into their own directories.

In [3]:
# Get a list of patient directories
patients=glob.glob(f'{path}/*')

# Set up target directories for file copy
idc_negative_directory='../data/idc_negative/'
idc_positive_directory='../data/idc_positive/'

Path(idc_negative_directory).mkdir(parents=True, exist_ok=True)
Path(idc_positive_directory).mkdir(parents=True, exist_ok=True)

# Loop on the patient directories and copy the '0' images to
# the idc_negative directory and the '1' images to the
# idc_positive directory
for patient in patients:

    # Copy the idc negative images from this patient
    idc_negative_images=glob.glob(f'{patient}/0/*.png')

    for image in idc_negative_images:
        shutil.copy(image, idc_negative_directory)

    # Copy the idc positive images from this patient
    idc_positive_images=glob.glob(f'{patient}/1/*.png')

    for image in idc_positive_images:
        shutil.copy(image, idc_positive_directory)

In [4]:
# Step 1b: Data Sampling for Efficiency
sampled_negative_directory = '../data_sampled/idc_negative/'
sampled_positive_directory = '../data_sampled/idc_positive/'

# Create directories for sampled data
Path(sampled_negative_directory).mkdir(parents=True, exist_ok=True)
Path(sampled_positive_directory).mkdir(parents=True, exist_ok=True)

# Sampling parameters
sample_size = 100  # Adjust as needed

In [5]:
# Function to sample and copy images
def sample_images(source_dir, target_dir, sample_size):
    images = glob.glob(f'{source_dir}/*.png')
    sampled_images = random.sample(images, min(sample_size, len(images)))
    for image in sampled_images:
        shutil.copy(image, target_dir)

In [ ]:
# Sample negative and positive images
sample_images(idc_negative_directory, sampled_negative_directory, sample_size)
sample_images(idc_positive_directory, sampled_positive_directory, sample_size)

print("Sampling complete!")

EDA

In [ ]:
idc_negative_count = len(glob.glob(f'{idc_negative_directory}/*.png'))
idc_positive_count = len(glob.glob(f'{idc_positive_directory}/*.png'))

print(f"IDC Negative Images: {idc_negative_count}")
print(f"IDC Positive Images: {idc_positive_count}")

In [ ]:
from tensorflow.keras.preprocessing import image

# Visualize IDC Negative and Positive Images Side-by-Side
negative_images = glob.glob(f'{idc_negative_directory}/*.png')[:3]
positive_images = glob.glob(f'{idc_positive_directory}/*.png')[:3]

fig, axs = plt.subplots(3, 2, figsize=(6, 6))  # Adjusted figsize for clarity
for neg_img, pos_img, row in zip(negative_images, positive_images, axs):
    for img_path, ax in zip([neg_img, pos_img], row):
        img = image.load_img(img_path)
        img = image.img_to_array(img)
        img /= 255.0  # Normalize image values
        ax.imshow(img)
        ax.axis('off')

plt.suptitle("IDC Negative (Left) vs. Positive (Right)")
plt.tight_layout()
plt.show()

In [ ]:
def plot_sample_images(directory, title, num_images=9):
    images = glob.glob(f"{directory}/*.png")
    fig, axs = plt.subplots(3, 3, figsize=(9, 9))
    for i, ax in enumerate(axs.flat[:min(num_images, len(images))]):
        img = plt.imread(images[i])
        ax.imshow(img)
        ax.axis('off')
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

plot_sample_images(idc_negative_directory, "IDC Negative Samples")
plot_sample_images(idc_positive_directory, "IDC Positive Samples")

In [ ]:
data_summary = pd.DataFrame({
    "Class": ["IDC Negative", "IDC Positive"],
    "Image Count": [idc_negative_count, idc_positive_count]
})
print(data_summary)

In [ ]:
# Check for class imbalance
total_images = idc_negative_count + idc_positive_count
print(f"Class Distribution:\n{data_summary}")
print(f"IDC Negative: {idc_negative_count / total_images:.2%}, IDC Positive: {idc_positive_count / total_images:.2%}")
if abs(idc_negative_count - idc_positive_count) / total_images > 0.2:
    print("Warning: Potential class imbalance detected.")

Prepare Image Dataset

In [ ]:
def make_datasets(training_data_path: str, image_dim: int, batch_size: int=32):
    training_dataset = tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='training',
        seed=315,
        image_size=(image_dim, image_dim),
        batch_size=batch_size,
        color_mode='grayscale'
    )
    validation_dataset = tf.keras.utils.image_dataset_from_directory(
        training_data_path,
        validation_split=0.2,
        subset='validation',
        seed=315,
        image_size=(image_dim, image_dim),
        batch_size=batch_size,
        color_mode='grayscale'
    )
    return training_dataset, validation_dataset

image_dim = 50
batch_size = 128
training_dataset, validation_dataset = make_datasets('../data/', image_dim, batch_size)

In [13]:
# training_dataset = training_dataset.take(100)
# validation_dataset = validation_dataset.take(50)

Sample Model

In [14]:
# def visualize_images(dataset, class_names):
#     images, labels = next(iter(dataset))
#     fig, axs = plt.subplots(3, 3, figsize=(9, 9))
#     for i, ax in enumerate(axs.flat[:images.shape[0]]):
#         ax.imshow(images[i].numpy().astype("uint8"))
#         ax.set_title(class_names[int(labels[i])])
#         ax.axis('off')
#     plt.tight_layout()
#     plt.show()

# class_names = ['IDC Negative', 'IDC Positive']
# visualize_images(training_dataset, class_names)

define the model

In [15]:
model = Sequential([
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

Compile the model

In [1]:
from tensorflow import keras

# Define the optimizer
optimizer=keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

2025-01-29 02:27:44.030066: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'model' is not defined

Train the model

In [ ]:
history = model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=50,
    steps_per_epoch=10,
    validation_steps=10
) 

Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(validation_dataset)
print(f"Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")

Training History

In [2]:
def plot_training_history(history):
    plt.figure(figsize=(12, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy Over Epochs')

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Over Epochs')

    plt.tight_layout()
    plt.show()

plot_training_history(history)

NameError: name 'history' is not defined

In [3]:
model.save('../data/model.keras')

NameError: name 'model' is not defined

In [21]:
import json

# Assume 'history' is the object returned by model.fit()
with open('../data/history.json', 'w') as f:
    json.dump(history.history, f)
